In [2]:
%config Completer.use_jedi = False

In [20]:
!pip install selenium webdriver-manager

In [18]:
import requests as req
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time
import random
import math
import json
import abc

print("Hello World")

Hello World


In [7]:
CHROM_DRIVE_PATH = "/workspace/chromedriver/133.0.6943.142/chromedriver.exe"
BASE_URL = "https://m.stock.naver.com/domestic/industry"

In [6]:
service = Service(CHROM_DRIVE_PATH)
driver = webdriver.Chrome(service=service)

actions = ActionChains(driver)

In [8]:
driver.get(BASE_URL)

In [29]:
class Stock(metaclass=abc.ABCMeta):    
    @abc.abstractmethod
    def get_categoty_list(self):
        pass
    
    @abc.abstractmethod
    def get_stock_list(self):
        pass
    
    @abc.abstractmethod
    def get_page_rate(self):
        pass

In [124]:
class UsaStocks(Stock):
    def __init__(self):
        self.BASE_URL = "https://m.stock.naver.com/worldstock/home/USA/industry"
    
    def get_categoty_list(self):
        driver.get(stock.BASE_URL)
        time.sleep(1 + random.random())

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        list_items = soup.select("ul.StockCategory_list__IuLu4 > li")
        return list(map(lambda x : x.text, list_items))

    def get_stock_list(self):
        def formatting(row):
            col_list = row.select("td")

            code = col_list[0].select("span")[0].text 

            company_name = col_list[0].select("span")[1].text

            # 현재가, 증감액
            current_price = float(col_list[1].select("span")[0].text.replace(",", ""))
            change_price = float(col_list[1].select("span")[1].select("span")[0].text.replace(",", ""))

            # 증감 비율
            change_rate = float(col_list[2].text[:-1])

            # 시가 총액, 거래량
            market_cap = int(col_list[3].text.replace(",", "") + "000")
            trade_value = col_list[4].text.replace(",", "")
            if(trade_value == "-"):
                trade_value = 0
            trade_value = int(trade_value)

            return {
                "code": code,
                "회사명": company_name,
                "현재가": current_price,
                "증감액": change_price,
                "증감비율": change_rate,
                "시가총액": market_cap,
                "거래량": trade_value,
            }

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)

        return list(map(formatting, soup.select("tr.TableComm_tr__QEe9V")))

    def get_page_rate(self):
        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        rate_num = soup.select(".VMore_link__Tsoh9 span")[1].text.split(" / ")
        return tuple(map(int, rate_num))
    
    def search(self):
        res = []
        categoty_list = self.get_categoty_list()
        category_count = len(categoty_list)

        for n, category in enumerate(categoty_list):
            progress_bar(n, category_count, category)
            list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{n + 1}]')
            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
            )
            list_itme.click()

            time.sleep(random.random())

            rate = 0, 1

            while(True):
                try:
                    reate = get_page_rate()
                    break
                except:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1 + random.random())


            for i in range(math.ceil(rate[1] / max(rate[0], 1))):
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    more_button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div[2]/div/div/div[10]/div/a')
                    actions.move_to_element(more_button).perform()
                    more_button.click()

                    time.sleep(0.1 + random.random())
                except:
                    time.sleep(2 + random.random())

            stock_list = get_stock_list()

            res.append({
                "카테코리" : category,
                "주식 목록" : stock_list,
            })

            driver.execute_script("window.scrollTo(0, 0);")

            prev_button = driver.find_element(by=By.XPATH, value='//*[@id="stockContentWrapper"]/div[8]/div/a')
            prev_button.click()

            time.sleep(1.5 + random.random())
        #     break
            if(n == 4):
                break
                
        return res

In [119]:
class KoreaStocks(Stock):
    def __init__(self):
        self.BASE_URL = "https://m.stock.naver.com/domestic/industry"
    
    def get_categoty_list(self):
        def formatting(tag):
            return tag.select("span")[0].text

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        list_items = soup.select("ul.GroupList_list__ICTC8 > li")
        return list(map(formatting, list_items))

    def get_stock_list(self):
        def formatting(row):
            col_list = row.select("td")

            code = col_list[0].select("span")[0].text 

            company_name = col_list[0].select("span")[1].text

            # 현재가, 증감액
            current_price = float(col_list[1].select("span")[0].text.replace(",", ""))
            change_price = float(col_list[1].select("span")[1].select("span")[0].text.replace(",", ""))

            # 증감 비율
            change_rate = float(col_list[2].text[:-1])

            # 시가 총액, 거래량
            market_cap = int(col_list[3].text.replace(",", "") + "000")
            trade_value = col_list[4].text.replace(",", "")
            if(trade_value == "-"):
                trade_value = 0
            trade_value = int(trade_value)

            return {
                "code": code,
                "회사명": company_name,
                "현재가": current_price,
                "증감액": change_price,
                "증감비율": change_rate,
                "시가총액": market_cap,
                "거래량": trade_value,
            }

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)

        return list(map(formatting, soup.select("tr.TableComm_tr__QEe9V")))

    def get_page_rate(self):
        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        rate_num = soup.select(".VMore_link__Tsoh9 span")[1].text.split(" / ")
        return tuple(map(int, rate_num))

In [14]:
def progress_bar(current, total, category, length=20):
    ratio = current / total  # 진행률 계산
    filled_blocks = int(length * ratio)  # ■ 개수
    empty_blocks = length - filled_blocks  # □ 개수
    bar = "■" * filled_blocks + "□" * empty_blocks
    percent = round(ratio * 100, 2)  # 퍼센트 표시 (소수점 2자리)
    print(f"[{bar}] {percent}% ({current}/{total}, {category})", end="\r")

In [125]:
stock = UsaStocks()

categoty_list = stock.search()

In [122]:
res = []

category_count = len(categoty_list)

for n, category in enumerate(categoty_list):
    progress_bar(n, category_count, category)
    list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{n + 1}]')
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
    )
    list_itme.click()
    
    time.sleep(random.random())
    
    rate = 0, 1
    
    while(True):
        try:
            reate = get_page_rate()
            break
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1 + random.random())
        
    
    for i in range(math.ceil(rate[1] / max(rate[0], 1))):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            more_button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div[2]/div/div/div[10]/div/a')
            actions.move_to_element(more_button).perform()
            more_button.click()

            time.sleep(0.1 + random.random())
        except:
            time.sleep(2 + random.random())
            
    stock_list = get_stock_list()
    
    res.append({
        "카테코리" : category,
        "주식 목록" : stock_list,
    })
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    prev_button = driver.find_element(by=By.XPATH, value='//*[@id="stockContentWrapper"]/div[8]/div/a')
    prev_button.click()
    
    time.sleep(1.5 + random.random())
#     break
    if(n == 4):
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="stockContentWrapper"]/div[8]/ul/li[1]"}
  (Session info: chrome=133.0.6943.142)
Stacktrace:
	GetHandleVerifier [0x00007FF75D35C685+28789]
	(No symbol) [0x00007FF75D2C5B00]
	(No symbol) [0x00007FF75D158F9A]
	(No symbol) [0x00007FF75D1AF346]
	(No symbol) [0x00007FF75D1AF57C]
	(No symbol) [0x00007FF75D202B17]
	(No symbol) [0x00007FF75D1D736F]
	(No symbol) [0x00007FF75D1FF7E3]
	(No symbol) [0x00007FF75D1D7103]
	(No symbol) [0x00007FF75D19FFC0]
	(No symbol) [0x00007FF75D1A1273]
	GetHandleVerifier [0x00007FF75D6A1ACD+3458237]
	GetHandleVerifier [0x00007FF75D6B827C+3550316]
	GetHandleVerifier [0x00007FF75D6ADB7D+3507565]
	GetHandleVerifier [0x00007FF75D422C4A+841274]
	(No symbol) [0x00007FF75D2D09CF]
	(No symbol) [0x00007FF75D2CCB14]
	(No symbol) [0x00007FF75D2CCCB6]
	(No symbol) [0x00007FF75D2BC0F9]
	BaseThreadInitThunk [0x00007FF9D5FF259D+29]
	RtlUserThreadStart [0x00007FF9D76E

In [258]:
list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{10}]')
driver.execute_script(
    "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
)
# list_itme.click()

In [279]:
with open("stock_data.json", "w", encoding="utf-8") as file:
    json.dump(res, file, ensure_ascii=False, indent=4)  